In [8]:
import requests as req
from bs4 import BeautifulSoup
import time

In [29]:
def scrapy_data():
    ''' get the responses from the concated urls
    
    returns:
        return the responses from the url,list
    '''
    
    ##concate the url to scrape
    responses = []
    url_port = 'http://kaijiang.500.com/shtml/ssq/'
    url_query = '.shtml?0_ala_baidu'
    minimum_weeks = 149
    from_year = '2005' ## 2 digits year
    for year in range(int(from_year),2020):
        for period in range(1,minimum_weeks):
            year_str = str(year)[-2:]
            if period < 10: ## period number <10
                period_str = '00'+str(period)
            elif period <100: ## period number <100
                period_str = '0' + str(period)
            else:            ## period number > 100
                period_str = str(period)
            ##concate the url
            url = url_port + year_str + period_str + url_query
            ## make the request
            time.sleep(5) ## slepp for a while
            try:
                yield req.get(url) ## make the request and have it back as an obj
                
            except :
                print('something wrong with connection')
#         print(f'{year} has been requested')

#     return responses

In [37]:
def scrapy_data(responses):
    '''scrapy data and yield a item
    return
        A well set of dictionary value container
    '''
    redbals = [] ## container to save the ball
    
    for res in responses:
        ##decode the byte content in a manner of chinese reconginzable
        res_text = res.content.decode('gb18030','ignore')
        soup = BeautifulSoup(res_text,'html.parser') ## parse the text to bs object
        ## get the red ball and blue ball
        balls_red = soup.find_all('li',attrs = {'class': 'ball_red'})
        for ball_red in balls_red:
            number = ball_red.text
            redbals.append(number)
        ball_blue = soup.find('li',attrs = {'class':'ball_blue'})
        ## get the date
        date = soup.find('td',attrs ={'class':'td_title01'}) ## father tag
        date_text = date.find('span',attrs ={'class':'span_right'}).text ## text
        open_date = date_text.split(' ')[0].split('：')[-1].replace('年','/').replace('月','/').replace('日','')
        close_dat = date_text.split('：')[-1].replace('年','/').replace('月','/').replace('日','')
        ## get money spend or deposited
        money = soup.find_all('span',attrs ={'class':'cfont1'})
        money_spend = money[0].text.replace('元','').replace(',','') ##spended
        money_deposit= money[1].text.replace('元','').replace(',','') ## deposited
        
        yield {
            'red_balls':redbals,
            'blue_balls':
        }
        
        

In [ ]:
def main():
    
    '''function to scrape the previous double
    color ball and generate a model to predict
    the exact number
    '''
    url = 'http://kaijiang.500.com/shtml/ssq/18148.shtml?0_ala_baidu'
    jakpot_number = {} ##in such a dict, peirod:{data:'',reward_num:'','deposited_from_last_period'}
    jakpot_agg = scrapy_data(url,jakpot_number) ## scrape the data from previous data
    jakpot_ax = plot_jakpot_agg(jakpot_agg) ## shown data with such plot
    jakpot_train,jakpot_valid,jakpot_test = preprocess(jakpot_agg) ## in a manner of preprocess,year,month,week and day
    jakpot_model = model_fit(jakpot_train,jakpot_valid) ## feed data to a neural network model and validated
    jakpot_model_evlued = jakpot_model_eva(jakpot_test) ## test data on the finally model